<a href="https://colab.research.google.com/github/qoyyimil/pba/blob/main/notebooks/1_Scraping_Moovit_Reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Scraping Google Play Store Reviews - Moovit (com.tranzmate)
Tugas 1A - Pemodelan Bahasa Alami

In [1]:
!pip install google_play_scraper
!pip install textblob
!pip install seaborn
!pip install langdetect

In [2]:
from google_play_scraper import app
import pandas as pd
import numpy as np
import sklearn
import requests
import matplotlib.pyplot as plt
import matplotlib.dates as dates
import seaborn as sns
import textblob
#from wordcloud import WordCloud
from pathlib import Path
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report, accuracy_score

import pickle
import re
import time
import datetime                              # access to %%time, for timing individual notebook cells
import os
from PIL import Image
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

%matplotlib inline
%config InlineBackend.figure_format='retina'

# Import seaborn styles explicitly
import seaborn as sns
# Apply the seaborn style before creating plots
sns.set_style("whitegrid")  # This line sets the Seaborn style

plt.rcParams["figure.figsize"] = (15,10)

In [6]:
# https://play.google.com/store/apps/details?id=com.tranzmate

from google_play_scraper import app, Sort, reviews_all
from langdetect import detect, DetectorFactory
from langdetect.lang_detect_exception import LangDetectException

# Memastikan hasil deteksi bahasa konsisten
DetectorFactory.seed = 0

mv_reviews = reviews_all(
    'com.tranzmate',
    sleep_milliseconds=0, # defaults to 0
    lang='id', # defaults to 'en'
    country='id',
    sort=Sort.NEWEST, # defaults to Sort.MOST_RELEVANT
)

In [7]:
# Save moovit reviews into dataframe
df_mvrev = pd.DataFrame(np.array(mv_reviews),columns=['content'])
df_mvrev = df_mvrev.join(pd.DataFrame(df_mvrev.pop('content').tolist()))

df_mvrev.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,8b6a424c-525c-40de-a4e1-e9976d9cba4f,Qurrota A'yun,https://play-lh.googleusercontent.com/a-/ALV-U...,"sangatttt sangattt membantuuuuuu, lope lope de...",5,0,5.176.0.1743,2025-09-12 08:26:29,None,NaT,5.176.0.1743
1,3865540d-065c-4b10-b7e3-9ddd55904703,ayu wandani,https://play-lh.googleusercontent.com/a/ACg8oc...,tidak membantu banyak iklan liat rute aja haru...,1,0,5.176.1.1744,2025-09-10 23:46:52,None,NaT,5.176.1.1744
2,991388ad-ac83-4a98-ac66-0d0b25ae1477,M. Eq Mero. S,https://play-lh.googleusercontent.com/a-/ALV-U...,uang ku kepotong gara gara aplikasi ini,1,0,None,2025-09-08 13:36:29,None,NaT,None
3,98bec169-47b5-4b9b-8190-c20bd3be3a47,Distorsi Pikiran,https://play-lh.googleusercontent.com/a-/ALV-U...,jujur kesel banget sama aplikasi ini. dulu bis...,1,4,5.176.0.1743,2025-09-04 23:40:46,None,NaT,5.176.0.1743
4,6ab59616-9392-4f1a-bb91-176a2ea80e3e,nurul istiqomah,https://play-lh.googleusercontent.com/a/ACg8oc...,sebenarnya sangat membantu tapi iklannya gangg...,3,5,5.175.2.1742,2025-09-04 05:54:59,None,NaT,5.175.2.1742


In [8]:
# Fungsi untuk mendeteksi bahasa dengan penanganan jika teks terlalu pendek
def detect_language(text):
    try:
        # Pastikan teks adalah string dan tidak kosong
        if isinstance(text, str) and len(text.strip()) > 10:
            return detect(text)
        else:
            return 'unknown'
    except LangDetectException:
        return 'error'

print("Memulai verifikasi bahasa pada kolom 'content'...")
# Terapkan fungsi pada setiap baris di kolom 'content'
df_mvrev['language'] = df_mvrev['content'].apply(detect_language)

print("\nDistribusi bahasa yang terdeteksi:")
print(df_mvrev['language'].value_counts())

# Filter DataFrame untuk hanya menyimpan review Bahasa Indonesia ('id')
df_indonesian = df_mvrev[df_mvrev['language'] == 'id'].copy()
# Hapus kolom 'language' karena tidak diperlukan lagi
df_indonesian.drop(columns=['language'], inplace=True)

print(f"\nJumlah review setelah difilter (hanya Bahasa Indonesia): {len(df_indonesian)} baris.")

Memulai verifikasi bahasa pada kolom 'content'...

Distribusi bahasa yang terdeteksi:
language
id         1843
unknown     702
tl           78
en           56
so           20
tr           18
hr           11
de           11
et           10
ro           10
da            8
lt            7
af            6
no            4
fi            4
cy            4
ca            3
hu            3
sw            2
nl            2
sl            2
lv            2
pt            2
error         1
sv            1
sq            1
ar            1
cs            1
fr            1
Name: count, dtype: int64

Jumlah review setelah difilter (hanya Bahasa Indonesia): 1843 baris.


In [9]:
# Censor usernames
def censor_username(username):
    if len(username) <= 2:
        return username
    else:
        return username[0] + '*' * (len(username) - 2) + username[-1]

df_mvrev['userName'] = df_mvrev['userName'].apply(censor_username)

In [10]:
# Save dataframe to csv
file_name = 'moovit_reviews_raw.csv'
df_mvrev.to_csv(file_name, index=False)

print(f"\n Data mentah berhasil disimpan ke dalam file: '{file_name}'")

df_mvrev.head()


 Data mentah berhasil disimpan ke dalam file: 'moovit_reviews_raw.csv'


,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion,language
0,8b6a424c-525c-40de-a4e1-e9976d9cba4f,Q***********n,https://play-lh.googleusercontent.com/a-/ALV-U...,"sangatttt sangattt membantuuuuuu, lope lope de...",5,0,5.176.0.1743,2025-09-12 08:26:29,None,NaT,5.176.0.1743,id
1,3865540d-065c-4b10-b7e3-9ddd55904703,a*********i,https://play-lh.googleusercontent.com/a/ACg8oc...,tidak membantu banyak iklan liat rute aja haru...,1,0,5.176.1.1744,2025-09-10 23:46:52,None,NaT,5.176.1.1744,id
2,991388ad-ac83-4a98-ac66-0d0b25ae1477,M***********S,https://play-lh.googleusercontent.com/a-/ALV-U...,uang ku kepotong gara gara aplikasi ini,1,0,None,2025-09-08 13:36:29,None,NaT,None,id
3,98bec169-47b5-4b9b-8190-c20bd3be3a47,D**************n,https://play-lh.googleusercontent.com/a-/ALV-U...,jujur kesel banget sama aplikasi ini. dulu bis...,1,4,5.176.0.1743,2025-09-04 23:40:46,None,NaT,5.176.0.1743,id
4,6ab59616-9392-4f1a-bb91-176a2ea80e3e,n*************h,https://play-lh.googleusercontent.com/a/ACg8oc...,sebenarnya sangat membantu tapi iklannya gangg...,3,5,5.175.2.1742,2025-09-04 05:54:59,None,NaT,5.175.2.1742,id
